In [1]:
import pandas as pd
import numpy as np
import tqdm
import matplotlib.pyplot as plt

In [2]:
# migrant_exc_all = pd.read_csv("Migrant/migrant_exc_all.csv")
# migrant_exc_all.drop(migrant_exc_all.columns[0],inplace=True,axis=1)

# migrant_exc_none = pd.read_csv("Migrant/migrant_exc_none.csv")
# migrant_exc_none.drop(migrant_exc_none.columns[0],inplace=True,axis=1)

migrant = pd.read_csv("Migrant/migrant_exc_all_new_1800.csv")
migrant.drop(migrant.columns[0],inplace=True,axis=1)

In [3]:
def migrants(language_out,language_in,years,exclude=""):
    migrant_df = pd.read_csv("Migrant/migrant_exc_all_new_1800.csv")
    migrant_df.drop(migrant_df.columns[0],inplace=True,axis=1)
    
    #Take only the first appearances of words
    first_appearance = migrant_df.drop_duplicates(subset='word', keep='first')
    
    #take the words created by the out language.
    out_words = first_appearance[first_appearance["language"] == language_out ]
    
    #Take the words that
    migrated_from_out = migrant_df[(migrant_df["word"].isin(out_words["word"])) & 
                               (migrant_df["year"].isin(years)) &
                                (migrant_df["language"] == language_in)
                               ]
    return(migrated_from_out)


In [6]:
migrants("italian","french",range(1850,2000),"_exc_all")

,word,year,count,language,Rank
249,agrippa,1881,10986,french,4799.0
254,agro,1980,17845,french,4690.0
770,armature,1891,13149,french,4365.0
1032,av,1944,5795,french,2804.0
1113,bagno,1938,9997,french,4324.0
...,...,...,...,...,...
13650,vaso,1875,11288,french,4205.0
13746,verbale,1980,16817,french,4938.0
13827,vico,1943,5150,french,3469.0
14262,xxii,1853,9357,french,4995.0


In [7]:
def uset(language_out,language_in,year,years_acc,lan_in_df,migrant_df):

    words = migrants(language_out,language_in,years_acc,exclude)


    
    lan_in_year = lan_in_df[lan_in_df["year"]==year].copy()

    lan_in_year['Rank'] = lan_in_year.groupby("year")["count"].rank(ascending=False, method='first')
    lan_in_year = lan_in_year.sort_values(by=["year", "Rank"])


    words_year = lan_in_year[lan_in_year["word"].isin(words["word"].values)]


    



    words_year = pd.merge(words_year,migrant_df,on="word",how="left")
    words_year = words_year[words_year["language"] == language_in]

    words_year.drop(columns = ["year_x","count_y","Rank_y","language"],inplace=True)
    words_year.rename(columns = {"count_x": "count "+ str(year), "Rank_x": "rank "+ str(year),
                            "year_y":"year of migration",
                                "word": "word "+ str(year)} ,inplace=True)

    words_year["rank " + str(year)] = words_year["rank "+ str(year)].astype(int)
    words_year = words_year.reset_index(drop=True)
    return(words_year)




In [9]:
language_out = "english"
language_in = "spanish"
exclude = "_exc_all"
year = 2009

lan_in_df = pd.read_csv("Top5000/"+"top_"+language_in + exclude+ ".csv",dtype={"0":str, "1":np.int32, "2":np.int32},
                         names = ["word","year","count"], header=None,index_col=False)


migrant_df = pd.read_csv("Migrant/migrant" + exclude + "_new_1800.csv")
migrant_df.drop(migrant_df.columns[0],inplace=True,axis=1)  

words_year = uset(language_out,language_in,year,range(1850,year+1),lan_in_df,migrant_df)

words_year

,word 2009,count 2009,rank 2009,year of migration
0,control,352106,76,1879
1,sector,219142,250,1926
2,university,117138,699,1916
3,similar,99536,885,1940
4,op,92831,979,1948
...,...,...,...,...
63,theory,20971,4916,2002
64,clinical,20882,4931,2009
65,terminal,20775,4963,2009
66,instrumental,20758,4967,2009


In [15]:
language_out = "italian"
language_in = "spanish"
# exclude = ""
languages = ["english","spanish","french","german","italian"]

exclude =  "_exc_all"
for language_out in languages:
    for language_in in languages:
        if language_out == language_in:
            continue
        
        

        lan_in_df = pd.read_csv("Top5000/"+"top_"+language_in + exclude+ ".csv",dtype={"0":str, "1":np.int32, "2":np.int32},
                         names = ["word","year","count"], header=None,index_col=False)
        
        migrant_df = pd.read_csv("Migrant/migrant" + exclude + "_new_1800.csv")
    
        migrant_df.drop(migrant_df.columns[0],inplace=True,axis=1) 


        result_df = uset(language_out,language_in,1900,range(1850,2009),lan_in_df,migrant_df)
        
        for year in tqdm.tqdm(range(1901,2009)):
            words_year = uset(language_out,language_in,year,range(1850,year+1),lan_in_df,migrant_df)
            result_df = pd.concat([result_df,words_year],axis=1)
            
        result_df.to_excel("Words/"+language_out+"_"+language_in+exclude+ "1800.xlsx",
                  index=False)

100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:03<00:00, 31.47it/s]


In [16]:
import openpyxl
exc = "_exc_all"

excel_files = []
for language_out in languages:
        for language_in in languages:
            if language_out == language_in:
                continue
            excel_files.append("Words/"+language_out+ "_"+ language_in+exc+"1800.xlsx")
            


# Create a new workbook to store merged sheets
merged_workbook = openpyxl.Workbook()

# Iterate through each Excel file
for file_index, excel_file in enumerate(excel_files):
    # Load the workbook
    wb = openpyxl.load_workbook(excel_file)
    for sheet_index, sheet_name in enumerate(wb.sheetnames):
        # Get the worksheet
        ws = wb[sheet_name]
        # Create a new sheet in the merged workbook
        new_sheet_name = excel_file[6:-5-len(exc)]
        new_sheet = merged_workbook.create_sheet(title=new_sheet_name)
        # Copy data from original sheet to the new sheet
        for row in ws.iter_rows(values_only=True):
            new_sheet.append(row)

# Save the merged workbook
merged_workbook.save("Migrated_Words_exc_all1800.xlsx")

In [11]:
1+1

2